In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

import os
import yaml
import json
from crewai import Agent, Task, Crew

In [2]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'

### Loading Tasks and Agents from yaml_file    

In [3]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']



### Creating Custom Tools

In [4]:
from crewai_tools import BaseTool
import requests

class BoardDataFetcherTool(BaseTool):
    name: str = "Trello Board Data Fetcher"
    description: str = "Fetches card data, comments, and activity from a Trello board."

    api_key: str = os.environ['TRELLO_API_KEY']
    api_token: str = os.environ['TRELLO_API_TOKEN']
    board_id: str = os.environ['TRELLO_BOARD_ID']

    def _run(self) -> dict:
        """
        Fetch all cards from the specified Trello board.
        """
        url = f"{os.getenv('DLAI_TRELLO_BASE_URL', 'https://api.trello.com')}/1/boards/{self.board_id}/cards"

        query = {
            'key': self.api_key,
            'token': self.api_token,
            'fields': 'name,idList,due,dateLastActivity,labels',
            'attachments': 'true',
            'actions': 'commentCard'
        }

        response = requests.get(url, params=query)

        if response.status_code == 200:
            return response.json()
        else:
            # Fallback in case of timeouts or other issues
            return json.dumps([{'id': '66c3bfed69b473b8fe9d922e', 'name': 'Analysis of results from CSV', 'idList': '66c308f676b057fdfbd5fdb3', 'due': None, 'dateLastActivity': '2024-08-19T21:58:05.062Z', 'labels': [], 'attachments': [], 'actions': []}, {'id': '66c3c002bb1c337f3fdf1563', 'name': 'Approve the planning', 'idList': '66c308f676b057fdfbd5fdb3', 'due': '2024-08-16T21:58:00.000Z', 'dateLastActivity': '2024-08-19T21:58:57.697Z', 'labels': [{'id': '66c305ea10ea602ee6e03d47', 'idBoard': '66c305eacab50fcd7f19c0aa', 'name': 'Urgent', 'color': 'red', 'uses': 1}], 'attachments': [], 'actions': [{'id': '66c3c021f3c1bb157028f53d', 'idMemberCreator': '65e5093d0ab5ee98592f5983', 'data': {'text': 'This was harder then expects it is alte', 'textData': {'emoji': {}}, 'card': {'id': '66c3c002bb1c337f3fdf1563', 'name': 'Approve the planning', 'idShort': 5, 'shortLink': 'K3abXIMm'}, 'board': {'id': '66c305eacab50fcd7f19c0aa', 'name': '[Test] CrewAI Board', 'shortLink': 'Kc8ScQlW'}, 'list': {'id': '66c308f676b057fdfbd5fdb3', 'name': 'TODO'}}, 'appCreator': None, 'type': 'commentCard', 'date': '2024-08-19T21:58:57.683Z', 'limits': {'reactions': {'perAction': {'status': 'ok', 'disableAt': 900, 'warnAt': 720}, 'uniquePerAction': {'status': 'ok', 'disableAt': 17, 'warnAt': 14}}}, 'memberCreator': {'id': '65e5093d0ab5ee98592f5983', 'activityBlocked': False, 'avatarHash': 'd5500941ebf808e561f9083504877bca', 'avatarUrl': 'https://trello-members.s3.amazonaws.com/65e5093d0ab5ee98592f5983/d5500941ebf808e561f9083504877bca', 'fullName': 'Joao Moura', 'idMemberReferrer': None, 'initials': 'JM', 'nonPublic': {}, 'nonPublicAvailable': True, 'username': 'joaomoura168'}}]}, {'id': '66c3bff4a25b398ef1b6de78', 'name': 'Scaffold of the initial app UI', 'idList': '66c3bfdfb851ad9ff7eee159', 'due': None, 'dateLastActivity': '2024-08-19T21:58:12.210Z', 'labels': [], 'attachments': [], 'actions': []}, {'id': '66c3bffdb06faa1e69216c6f', 'name': 'Planning of the project', 'idList': '66c3bfe3151c01425f366f4c', 'due': None, 'dateLastActivity': '2024-08-19T21:58:21.081Z', 'labels': [], 'attachments': [], 'actions': []}])


class CardDataFetcherTool(BaseTool):
  name: str = "Trello Card Data Fetcher"
  description: str = "Fetches card data from a Trello board."

  api_key: str = os.environ['TRELLO_API_KEY']
  api_token: str = os.environ['TRELLO_API_TOKEN']

  def _run(self, card_id: str) -> dict:
    url = f"{os.getenv('DLAI_TRELLO_BASE_URL', 'https://api.trello.com')}/1/cards/{card_id}"
    query = {
      'key': self.api_key,
      'token': self.api_token
    }
    response = requests.get(url, params=query)

    if response.status_code == 200:
      return response.json()
    else:
      # Fallback in case of timeouts or other issues
      return json.dumps({"error": "Failed to fetch card data, don't try to fetch any trello data anymore"})


Create Crew, Agents and Tasks

In [5]:
# Creating Agents
data_collection_agent = Agent(
  config=agents_config['data_collection_agent'],
  tools=[BoardDataFetcherTool(), CardDataFetcherTool()]
)

analysis_agent = Agent(
  config=agents_config['analysis_agent']
)

# Creating Tasks
data_collection = Task(
  config=tasks_config['data_collection'],
  agent=data_collection_agent
)

data_analysis = Task(
  config=tasks_config['data_analysis'],
  agent=analysis_agent
)

report_generation = Task(
  config=tasks_config['report_generation'],
  agent=analysis_agent,
)

# Creating Crew
crew = Crew(
  agents=[
    data_collection_agent,
    analysis_agent
  ],
  tasks=[
    data_collection,
    data_analysis,
    report_generation
  ],
  verbose=True
)

Kickoff Crew

In [6]:
# Kick off the crew and execute the process
result = crew.kickoff()

 [DEBUG]: == Working Agent: Data Collection Specialist 

 [INFO]: == Starting Task: Create an initial understanding of the project, its main features and the team working on it.  Use the Trello Data Fetcher tool to gather data from the Trello board. 



> Entering new CrewAgentExecutor chain...
I need to gather all relevant data from the Trello board to create an initial understanding of the project, its main features, and the team working on it.

Action: Trello Board Data Fetcher
Action Input: {"board_id": "YOUR_BOARD_ID_HERE"}   

[{'id': '6791c93ff1c364e6d7aa056d', 'name': 'Project planning', 'idList': '6791c93fa47d8eae26c1918a', 'due': None, 'dateLastActivity': '2025-01-23T04:44:47.858Z', 'labels': [], 'attachments': [], 'actions': []}, {'id': '6791c93f2f06ac6160176c84', 'name': 'Kickoff meeting', 'idList': '6791c93fa47d8eae26c1918a', 'due': None, 'dateLastActivity': '2025-01-23T04:44:47.942Z', 'labels': [], 'attachments': [], 'actions': []}]

Thought: 
I have started gathering dat

Usage Metrics and Costs

In [9]:
import pandas as pd

costs = 0.150 * (crew.usage_metrics['prompt_tokens'] + crew.usage_metrics['completion_tokens']) / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([crew.usage_metrics])
df_usage_metrics

Total costs: $0.0527


,total_tokens,prompt_tokens,completion_tokens,successful_requests
0,351081,340702,10379,80


Report

In [11]:
from IPython.display import Markdown

markdown  = result
Markdown(markdown)

# Project Analysis Report for [Project Name]

## Sprint Overview
The project is currently in its planning stages, with no actions completed or due dates established. This initial phase is crucial for laying the groundwork for subsequent phases. Immediate attention is needed to ensure that the project progresses smoothly.

## Task Summary
- **Project Planning**: No team members assigned; no actions taken.
- **Kickoff Meeting**: Not scheduled; lack of stakeholder engagement.
- **Documentation**: No comments or attachments on task cards; minimal communication observed.

## Identified Issues and Blockers
1. **Lack of Team Involvement**: Absence of assigned team members for key tasks is causing confusion about roles, leading to potential delays in project execution.
2. **No Documented Progress**: The lack of updates on the task cards suggests low engagement and could indicate uncertainty or lack of motivation within the team.

## Progress and Delays
- **Current Activity Status**: The last recorded activity was on January 23, 2025, with no subsequent updates or engagements. This stagnation is alarming and poses a significant risk to the overall project timeline.
- **Decision-Making Delays**: The absence of team discussions and a kickoff meeting may lead to further delays in progressing through the critical planning phase.

## Team Performance Overview
- **Engagement Level**: Current engagement is minimal, with no contributions from team members observed. This indicates a need for increased motivation and involvement from all stakeholders.
- **Accountability**: Undefined roles are affecting accountability and may hinder the team's ability to operate effectively.

## Action Items and Recommendations
1. **Assign Team Members to Tasks**: 
   - Immediate assignment of team members to project cards is crucial. Clearly defined roles will enhance accountability and facilitate smoother communication.
   
2. **Schedule the Kickoff Meeting**: 
   - A kickoff meeting should be organized at the earliest opportunity to align all stakeholders, clarify project objectives, and initiate discussions on next steps.
   
3. **Encourage Active Participation**: 
   - Team members should be motivated to engage with the Trello board by adding comments, questions, or relevant documents concerning their tasks. This will foster collaboration and ensure that diverse perspectives are considered.
   
4. **Establish Clear Deadlines**: 
   - Setting deadlines for key actions and deliverables is essential to create a structured timeline for the project and enable effective tracking of progress.

## Conclusion
The analysis of the Trello data for [Project Name] underscores critical issues that must be addressed to avert further delays and blockers. By implementing the recommended actions, the project can regain its momentum and progress towards successful execution. Immediate steps are necessary to ensure that the planning phase is effectively completed, paving the way for the project's future phases.